Ignis: Latent Dirichlet Allocation
============

In [1]:
import ignis

Model training (LDA)
----

Load from an `ignis.Corpus`, add the processed docs to an LDA model, and train it.

The random seed and parallelisation can both affect results, so setting the seed and number of workers is necessary for reproducibility.

In [2]:
corpus = ignis.load_corpus("bbc.corpus")

In [3]:
model_options = {"term_weighting": "pmi", "verbose": True, "workers": 10, "until_max_coherence": True}
vis_options = {"verbose": True}

In [4]:
results = ignis.train_model(
    corpus,
    model_options=dict(model_options, k=5),
)

Training model on 2118 documents:
{'term_weighting': 'pmi', 'k': 5, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -2.18493 (Starting: -2.23163)
Docs: 2118, Vocab size: 19290, Total Words: 412636
Model training complete. (22.960s)


In [5]:
results.init_vis("pyldavis")
results.show_visualisation()

In [6]:
# results.export_visualisation("bbc_results")

## Saving results, using the automated labeller

In [7]:
results.save("bbc.aurum")
results = ignis.load_results("bbc.aurum")

In [8]:
results.init_labeller("tomotopy", verbose=True)

Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


## Exploring documents that "belong" to a given topic

Because topics are distributions over words and documents are *distributions* over topics, documents don't belong to individual topics per se; every topic is represented in every document with some probability.

We therefore have to specify how many of the document's top `n` topics we want to check for the actual topic we're interested in.

This is especially significant for topic models that use a term weighting scheme, because all the common words (i.e., what we might consider stopwords) tend to get grouped into a single large topic; if we only consider each document's single most probable topic, we will unintentionally exclude documents which have this "stopwords" topic as their top topic.  (This is, however, not always a given; one would need to check the model's output and iterate from there).

Alternatively, we can specify `ignore_topics` when slicing to completely ignore a given topic when classifying documents.

In [9]:
results.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=5, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>

Slicing and iteration
--------
After seeing what the main topics might be, we can slice the initial corpus further and re-run topic modelling to get better resolution.

In [10]:
# Try zooming in on the entertainment topic
entertainment_slice = results.slice_by_topic(topic_id=3)

We might start by checking exactly which documents a slice of the corpus contains, by exploring it directly:

In [13]:
entertainment_slice.nb_explore(doc_sort_key=lambda doc: doc.metadata["filename"])

interactive(children=(IntSlider(value=0, description='Document', max=331), Output()), _dom_classes=('widget-in…

<function ignis.corpus.CorpusSlice.nb_explore.<locals>.show_doc(index=0)>

Ignis can suggest a recommended number of topics to use for LDA, based on the coherence scores of a range of lightly-trained models.

In [16]:
# Getting a suggested number of topics and retraining the model on the new slice
best_k = results.resuggest_num_topics(corpus_slice=entertainment_slice, start_k=3, verbose=True)
entertainment_model = results.retrain_model(
    corpus_slice=entertainment_slice, model_options={"k": best_k}
)
entertainment_model.show_visualisation()

Training 8 mini-models to suggest a suitable number of topics between 3 and 10...(332 documents, 100 iterations each)


Suggested topic count: 6	Coherence: -2.4081787306367293 (always negative, closer to 0 is better)

Training model on 332 documents:
{'term_weighting': 'pmi', 'k': 6, 'seed': 11399, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -4.68509 (Starting: -4.70856)
Docs: 332, Vocab size: 8667, Total Words: 54872
Model training complete. (5.409s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


The position of each topic cluster on the graph is not intrinsically informative per se (being simply the result of some specified dimensionality-reducing technique), but if we want we can run the modelling algorithm with a different random seed and see if we get a more meaningful set of topics.

(We can also override any of the previously set options.)

In [17]:
new_model_options = {"seed": 7156}
entertainment_model_2 = entertainment_model.retrain_model(
    model_options=new_model_options
)
entertainment_model_2.show_visualisation()

Training model on 332 documents:
{'term_weighting': 'pmi', 'k': 6, 'seed': 7156, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -5.12167 (Starting: -5.32957)
Docs: 332, Vocab size: 8667, Total Words: 54872
Model training complete. (5.465s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


Notice that the topic words remain more or less consistent across different training runs, even though their positions in the visualisation change with each iteration.

In [18]:
entertainment_model_2.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=6, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>

In addition to simply slicing by topic, we can also explicitly search the whole corpus for documents that contain certain tokens, in case we want to be absolutely sure we got all the documents that mention certain words or phrases.

In [19]:
# Topics that seem to deal with films
film_topic = entertainment_model_2.slice_by_topics(topic_ids=[4,5])

# Tokens that are related to films, doing a full-text search through the entire corpus (not just within the current results)
by_token_slice = entertainment_model_2.slice_by_tokens(
    ["film", "films", "movie", "cinema"], include_root=True
)

concat_slice = film_topic.concat(by_token_slice)

concat_model = entertainment_model_2.retrain_model(
    concat_slice, model_options=model_options
)
concat_model.show_visualisation()

Training model on 309 documents:
{'term_weighting': 'pmi', 'k': 6, 'seed': 7156, 'workers': 10, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'until_max_coherence': True, 'max_extra_iterations': 2000, 'verbose': True, 'tw': <TermWeight.PMI: 2>, 'parallel': <ParallelScheme.DEFAULT: 0>}





Continuing to train until maximum coherence.




Best coherence: -3.20616 (Starting: -3.20616)
Docs: 309, Vocab size: 9680, Total Words: 63422
Model training complete. (4.742s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


In [20]:
# Topic 2 seems related to film awards
concat_model.nb_explore_topics()

interactive(children=(IntSlider(value=1, description='Topic', max=6, min=1), Output()), _dom_classes=('widget-…

<function ignis.aurum.Aurum.nb_explore_topics.<locals>.show_topic(topic_id=1)>